# importing packages

In [44]:
!pip install git+https://github.com/LIAAD/yake

  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-i7_1rztx
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-i7_1rztx
     |████████████████████████████████| 102kB 7.5MB/s 
  Created wheel for yake: filename=yake-0.4.4-py2.py3-none-any.whl size=59909 sha256=3b51a598fc956901f2002201bf6e46036818a7c264e35faacd1cd7a3f089910c
  Stored in directory: /tmp/pip-ephem-wheel-cache-zdhincb6/wheels/be/35/27/e4ebd54b78c1806ed8b0271ce247fcd91e2bedde35889fbc9b
  Created wheel for segtok: filename=segtok-1.5.10-cp36-none-any.whl size=25019 sha256=77635b5859b00d003d8e8dc39bd776bb1a0d47d74ce56606b0fd8922eef2730c
  Stored in directory: /root/.cache/pip/wheels/b4/39/f6/9ca1c5cabde964d728023b5751c3a206a5c8cc40252321fb6b
Successfully built yake segtok


In [43]:
#pip install --upgrade torch torchvision

In [2]:
!pip install -qU sentence-transformers
!pip install -qU wikipedia-api
!pip install -qU clean-text[gpl]

!mkdir resources
!wget -q "https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip" -P resources
!unzip -qq resources/resources-0.5.zip -d resources

!rm -rf /content/4ccae468eb73bf6c4f4de3075ddb5336
!rm -rf /content/preproc
!rm preprocessing.py utils.py
!mkdir -p /content/preproc
!git clone https://gist.github.com/4ccae468eb73bf6c4f4de3075ddb5336.git /content/preproc/
!mv /content/preproc/* /content/
!rm -rf /content/preproc

!pip install faiss-cpu
!pip install hazm

     |████████████████████████████████| 71kB 5.0MB/s 
     |████████████████████████████████| 1.8MB 7.8MB/s 
     |████████████████████████████████| 1.2MB 38.3MB/s 
     |████████████████████████████████| 3.2MB 49.3MB/s 
     |████████████████████████████████| 890kB 56.6MB/s 
     |████████████████████████████████| 133kB 8.7MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 245kB 13.7MB/s 
rm: cannot remove 'preprocessing.py': No such file or directory
rm: cannot remove 'utils.py': No such file or directory
Cloning into '/content/preproc'...
remote: Enumerating objects: 7, done.
remote: Total 7 (delta 0), reused 0 (delta 0), pack-reused 7
Unpacking objects: 100% (7/7), done.
     |████████████████████████████████| 8.2MB 6.0MB/s 
     |████████████████████████████████| 317kB 7.7MB/s 
     |████████████████████████████████| 1.4MB 13.2MB/s 
     |████████████████████████████████| 235kB 34.0MB/s 
  Created wheel for nltk: filename=nltk-3.3-c

In [45]:
import numpy as np 
import pandas as pd
import re
import os
import faiss
import hazm
from hazm import stopwords_list
import pickle
import requests
from termcolor import colored

from torch.utils.data import DataLoader
from sentence_transformers import InputExample, losses
import torch
import tensorflow as tf
from sentence_transformers import models, SentenceTransformer, util, evaluation
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics.pairwise import cosine_similarity
import yake

SentenceTransformer_TrainedOn_FarsTail_Path = 'm3hrdadfi/bert-fa-base-uncased-farstail-mean-tokens'
SentenceTransformer_TrainedOn_wikinli_Path = 'm3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens'
SentenceTransformer_TrainedOn_wikitriplet_Path = 'm3hrdadfi/bert-fa-base-uncased-wikitriplet-mean-tokens'

# The Dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:

data_address = '/content/drive/MyDrive/COVID-PSS.xls'
keys_address = '/content/drive/MyDrive/keywords_final_distilled_NE (1).pickle'
cleaned_titles_address = '/content/drive/MyDrive/title_cleaned_without_corona_2.pkl'

df = pd.read_csv(data_address)
list_t = pd.read_pickle(cleaned_titles_address)

keywords = pd.read_pickle(keys_address)
keywords = [v for k,v in keywords.items()]



assert len(keywords) == len(df)
df['keywords'] = keywords
df.drop(columns=['img', 'link'], inplace=True)

# preparing inputs for semantic

corpora = []
for i in range(len(list_t)):

    keys = '[SEP]'.join(keywords[i])
    corpora.append(' '.join([list_t[i], keys]))


keys = []
for i in range(len(list_t)):
    keys.append('[SEP]'.join(keywords[i]))

bm_selected = pd.read_pickle('/content/drive/MyDrive/CoPer paper-Models/Results/BM25_PrimaryRanking.pkl')

# annotations

df_annotations = pd.read_csv('/content/drive/MyDrive/all_annotations.csv')
df_annotations.loc[:, 'score'] = df_annotations.loc[:, 'score'].apply(lambda x: x-1)
print(f'minimum score: {min(df_annotations.score)} and maximum: {max(df_annotations.score)}')

minimum score: 0.0 and maximum: 4.0


In [ ]:
df_annotations

,score,sentence1,sentence2
0,3.5,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...,گروهی از پسران در حیاط بازی می کنند و مردی در ...
1,2.2,گروهی از کودکان در خانه مشغول بازی هستند و هیچ...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...
2,3.7,پسران جوان در فضای باز بازی می کنند و مرد در ه...,بچه ها در بیرون از خانه و در کنار یک مرد با لب...
3,2.4,بچه ها در بیرون از خانه و در کنار یک مرد با لب...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...
4,2.7,پسران جوان در فضای باز بازی می کنند و مرد در ه...,گروهی از بچه ها در حیاط بازی می کنند و پیرمردی...
...,...,...,...
2738,2.4,یک سگ قهوه ای در حال عبور از یک مسیر برفی است,یک سگ سفید از مسیر برفی در حال دویدن است
2739,3.5,یک سگ قهوه ای در حال عبور از یک مسیر برفی است,سگی در مسیری برفی در حال دویدن است
2740,3.3,یک سگ سفید از مسیر برفی در حال دویدن است,سگی در مسیری برفی در حال دویدن است
2741,2.4,سگی در مسیری گل آلود در حال دویدن است,یک سگ قهوه ای در حال عبور از یک مسیر برفی است


# Helpers

In [6]:

# phrases that need to be removed from titles
corona_phrases = ['کرونایی', 'کروناست' ,'کرونا', 'شیوع', 'بحران', 'ویروس',
                  'ویروس جدید', 'coronavirus', 'corona', 'کووید-19 ', 
                  'کووید', 'بیماری', 'بیمارانی', 'بیماران', '-۱۹', ' وی ', '19', '۱۹',
                  ' بیمار ', 'كرونا', 'كوويد', 'ويروس', r'(\s+)',]


normalizer = hazm.Normalizer()

def clean(text):
    """Cleans the titles for the semantic models"""
    for pattern in corona_phrases:
        text = re.sub(pattern, " ", text)

    text = re.sub(' +[\w] +', " ", text)
    text = normalizer.normalize(text)

    return text


In [7]:

def get_resutls(bm_selected, title_to_keys_ratio=1.2):
    
    results = []
    for i in range(len(bm_selected)):
        # the question we're getting the similars for
        print(bm_selected[i]['question'])
        question = bm_selected[i]['question']

        # we give the cleaned question to the semantic model 
        question_cleaned = clean(question)
        
        # indices selected by bm25 for this question
        bm_indices = bm_selected[i]['index']

        # getting the specific indices embeddings for keys and titles
        keys_ranked = [key_embeddings[j] for j in bm_indices]
        list_t_ranked = [title_embeddings[j] for j in bm_indices]
        
        # taking the weighted average of keys and title for all selected records
        each_record_value = [(key_embeddings[m] + title_embeddings[m] *title_to_keys_ratio)/(title_to_keys_ratio+1) for m in range(len(bm_indices))]
        
        question_emb = sentence_bert_embedder.encode(question_cleaned,
                                                    convert_to_tensor=False,
                                                    show_progress_bar = False)
                        
        #----------------------faiss for the selected corpora----------------------------------#                
        index = faiss.index_factory(768, "Flat" , faiss.METRIC_INNER_PRODUCT)
        
        # corpora
        emb_cor = np.array([n.numpy() for n in each_record_value])
        faiss.normalize_L2(emb_cor)
        index.add(emb_cor)

        # question
        emb_que = np.array([question_emb])
        faiss.normalize_L2(emb_que)

        #getting top n results
        top_k = index.search(emb_que, len(bm_indices))
        indices = []
        scores = []

        # saving all the reults in a dictionary
        for score, idx in zip(top_k[0][0], top_k[1][0]):
            
            indices.append(idx)
            scores.append(score)

        results.append({'question':question,
                        'index':indices,
                        'score':scores})
        
    return results


# The model


In [8]:
model = SentenceTransformer(SentenceTransformer_TrainedOn_wikinli_Path)

Exception when trying to download https://sbert.net/models/m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens.zip. Response 404
SentenceTransformer-Model https://sbert.net/models/m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens.zip not found. Try to create it from scratch
Try to create Transformer Model m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens with mean pooling


In [19]:
epochs = 1
warm_up_s = 50
number_of_records=300

# Fine-tuning

In [28]:


df_annotations_train = df_annotations.copy()[:number_of_records] # only a part of dataframe

labels_for_kfold = df_annotations_train.score.apply(lambda x: round(x))# for letting sklearn understand different labels


df_annotations_train.loc[:, 'score'] = df_annotations_train.score.apply(lambda x:x/4) # for cosine similarity




In [32]:

k = 3
kfold = StratifiedKFold(n_splits = k, shuffle = True)

for i, (train_idx, val_idx) in enumerate(kfold.split(df_annotations_train.sentence2, labels_for_kfold), 1):
    print(colored(f'fold {i}', 'blue'))
    epoch_evaluation = {}
    print(train_idx)

    # then each in an example
    training_examples = []
    for _, record in df_annotations_train.loc[train_idx].iterrows():
        training_examples.append(InputExample(texts = [str(record.sentence1),
                                                       str(record.sentence2)],
                                               label= np.float(record.score)))


    # using dataloader to give data to our model
    training_dataloader = DataLoader(training_examples,
                                    shuffle = True,
                                    batch_size = 128)

    loss_ = losses.CosineSimilarityLoss(model)

    # print(df_annotations_train.loc[val_idx, 'sentence1'])


    evaluator = evaluation.EmbeddingSimilarityEvaluator(df_annotations_train.loc[val_idx, 'sentence1'].values.tolist(),
                                                        df_annotations_train.loc[val_idx, 'sentence2'].values.tolist(),
                                                        df_annotations_train.loc[val_idx, 'score'].values.tolist())


    %time
    model.fit(
        train_objectives=[(training_dataloader, loss_)],
        epochs=epochs,
        evaluator=evaluator, 
        #warmup_steps=warm_up_s, 
        # write_csv =True
        )

fold 1
[  1   2   6   7   8  10  12  13  14  15  16  17  19  20  22  24  28  30
  32  33  34  35  38  39  44  45  47  49  50  51  52  53  54  55  56  58
  60  61  63  64  66  69  70  71  72  73  74  75  77  78  79  80  82  83
  84  87  89  91  92  93  95  96  97  98  99 100 101 102 104 105 109 110
 111 113 115 118 119 120 122 123 125 126 130 133 134 135 136 137 138 139
 140 141 142 143 144 147 149 150 152 153 154 155 156 159 160 163 165 166
 169 170 171 172 173 174 177 178 179 180 181 183 185 187 189 191 192 193
 195 196 200 201 202 204 205 206 207 208 210 211 215 217 218 219 220 221
 222 223 225 226 228 229 230 231 232 234 235 238 242 245 247 248 250 252
 253 255 256 257 259 260 261 262 263 264 265 266 267 268 269 271 272 273
 274 275 276 277 278 279 280 281 282 283 284 285 286 287 290 291 292 294
 295 298]
CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.77 µs




fold 2
[  0   1   3   4   5   7   8   9  10  11  13  14  15  16  17  18  21  22
  23  24  25  26  27  29  31  34  36  37  38  40  41  42  43  46  47  48
  50  51  53  55  56  57  58  59  62  63  65  67  68  76  78  81  83  84
  85  86  87  88  90  92  93  94  96  97  98 102 103 106 107 108 110 112
 114 115 116 117 119 120 121 124 127 128 129 130 131 132 134 136 139 140
 141 142 145 146 147 148 149 150 151 152 153 155 157 158 159 160 161 162
 163 164 165 166 167 168 169 171 172 173 175 176 179 181 182 183 184 186
 188 190 193 194 197 198 199 200 201 203 204 209 210 212 213 214 216 217
 218 219 220 221 222 223 224 225 227 228 229 230 232 233 235 236 237 238
 239 240 241 243 244 246 247 248 249 251 252 253 254 255 256 258 261 267
 268 269 270 274 276 278 279 281 283 286 288 289 290 291 293 294 295 296
 297 299]
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.53 µs




fold 3
[  0   2   3   4   5   6   9  11  12  18  19  20  21  23  25  26  27  28
  29  30  31  32  33  35  36  37  39  40  41  42  43  44  45  46  48  49
  52  54  57  59  60  61  62  64  65  66  67  68  69  70  71  72  73  74
  75  76  77  79  80  81  82  85  86  88  89  90  91  94  95  99 100 101
 103 104 105 106 107 108 109 111 112 113 114 116 117 118 121 122 123 124
 125 126 127 128 129 131 132 133 135 137 138 143 144 145 146 148 151 154
 156 157 158 161 162 164 167 168 170 174 175 176 177 178 180 182 184 185
 186 187 188 189 190 191 192 194 195 196 197 198 199 202 203 205 206 207
 208 209 211 212 213 214 215 216 224 226 227 231 233 234 236 237 239 240
 241 242 243 244 245 246 249 250 251 254 257 258 259 260 262 263 264 265
 266 270 271 272 273 275 277 280 282 284 285 287 288 289 292 293 296 297
 298 299]
CPU times: user 1 µs, sys: 1e+03 ns, total: 2 µs
Wall time: 4.53 µs


In [33]:
def create_custome_sentencebert(model):
    """
    adds a pooler at the end of the fine-tuned model
    """
    bert_model_s = model
    pooling_model = models. Pooling(
        768,
        pooling_mode_mean_tokens = True,
        pooling_mode_cls_token = False,
        pooling_mode_max_tokens = False,
    )
    model = SentenceTransformer(modules= [bert_model_s, pooling_model])
    return model

sentence_bert_embedder = create_custome_sentencebert(model = model)

In [34]:
key_embeddings = sentence_bert_embedder.encode(keys,
                                                convert_to_tensor=True,
                                                show_progress_bar=True,
                                                batch_size = 256 # 128 previously
                                                )

title_embeddings = sentence_bert_embedder.encode(list_t,
                                                  convert_to_tensor=True,
                                                  show_progress_bar=True,
                                                  batch_size = 256 # 128 previously
                                                  )

In [ ]:
language = "fa"
max_ngram_size = 2 # max # of words in each keyword
deduplication_thresold = 0.9 # how much they are allowed to overlap
deduplication_algo = 'seqm'
windowSize = 1 # how many words to consider when calculating the importance of 
               # each word regarding its surroundings
features = None # Whether or not to calculate a specific measurement for
                # term and word frequency
numOfKeywords = 1


def extract_keys(corpus, numOfKeywords):
    kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=features)
    keywords = kw_extractor.extract_keywords(corpus)
    # the lower the score the more relevant the key
    keys = [key[0] for key in sorted(keywords, key =  lambda x: x[1])]
    return keys

In [47]:
results = get_resutls(bm_selected, title_to_keys_ratio=1.2)

آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند 
من باردار هستم چگونه می‌توانم از خودم در مقابل بیماری کرونامحافظت کنم
آیا زنان باردار باید برای کروناآزمایش شوند
آیا کرونامی تواند از مادر باردار به نوزاد متولد نشده جنین یا نوزاد تازه متولد شده منتقل شود
در دوران بارداری حین زایمان چه مراقبت‌هایی باید انجام شود
آیا زنان باردار مبتلا به کرونایا مشکوک به این بیماری نیاز به سزارین دارند
آیا مادر مبتلا به کرونامی تواند به نوزاد خود شیر بدهد
آیا در صورت مبتلا شدن به کووید۱۹ می‌توانم کودک خود را در آغوش بگیرم
من به کرونامبتلا هستم حال عمومی مساعدی ندارم که بتوانم به کودک خود مستقیما شیر بدهم چه باید بکنم
آیا خشک­ کن­های دست در نابودی ویروس جدید کرونا موثر هستند
آیا گندزداییضدعفونی با لامپ ماورای بنفش ویروس جدید کرونا را از بین می­برد
آیا سنجش حرارت بدن اندازه گیری تب در شنسایی افراد مبتلا به عفونت ویروس جدید کرونا موثر است
آیا اسپری کردن الکل یا محلول‌های حاوی کلر روی تمام بدن می­تواند ویروس جدید کرونا را نابود کند
آیا واکسنهای ذات­الریه در برابر ویروس جدید کرونا موثر هستند
آیا ش

In [ ]:
assert len(results) == len(bm_selected)
assert [que['question'] for que in results] == [bm['question'] for bm in bm_selected]
print(len(results[0]['index']))

3517


In [ ]:
len(bm_selected[0]['index'])

3517

In [36]:
# taking the weighted average
title_to_keys_ratio = 1.2
each_record_value = [(key_embeddings[i] + title_embeddings[i] *title_to_keys_ratio)/(title_to_keys_ratio+1) for i in range(len(corpora))]

In [40]:
question = 'آیا زنان باردار بیشتر در معرض خطر ابتلا به بیماری کروناهستند '
question_emb = sentence_bert_embedder.encode(question,
                               convert_to_tensor=False,
                               show_progress_bar = False)

In [41]:
index = faiss.index_factory(768, "Flat", faiss.METRIC_INNER_PRODUCT)
emb_cor = np.array([i.numpy() for i in each_record_value])
faiss.normalize_L2(emb_cor)
index.add(emb_cor)

emb_que = np.array([question_emb])
faiss.normalize_L2(emb_que)

top_k = index.search(emb_que, 50)

In [42]:
sbert_different_weights_indices = []
sbert_different_weights_scores = []


print('Top results!')
for score, idx in zip(top_k[0][0], top_k[1][0]):
    print(colored(f'{idx}th', 'blue'),' corpus with score', colored( f'{score:.2f}:\n', 'blue'),  corpora[idx])
    sbert_different_weights_indices.append(idx)
    sbert_different_weights_scores.append(score)

Top results!
2251th  corpus with score 0.93:
 سیگاری‌ها بیشتر در خطر ابتلا به هستند معرض ابتلا[SEP]درگیر میکند[SEP]مصرف دخانیات
1865th  corpus with score 0.91:
 خطر ابتلا به سکته مغزی در افراد سیگاری مبتلا به بیشتر است لخته شدن خون[SEP]دلیل لخته شدن خون[SEP]سیگار کشیدن میتواند شدت cov id[SEP]التهاب مغز اسیب عصبی و سکته مغزی[SEP]سیگار کشیدن خطر ابتلا به[SEP]سکته مغزی[SEP]دلیل افزایش عوامل لخته شدن خون[SEP]سد خونی مغزی[SEP]خطر لخته شدن خون[SEP]خطر سکته مغزی[SEP]دلیل اسیب عروق خونی و خطر سکته مغزی[SEP]سکته مغزی ناگهانی[SEP]باعث افزایش پروتیینهای لخته شدن خون[SEP]میزان عوامل لخته شدن خون[SEP]سیگار کشیدن خطر سکته مغزی[SEP]۲ شدت cov id[SEP]سیگار کشیدن
2035th  corpus with score 0.91:
 خطر مرگ ابتلا به در کودکان چاق بیشتر است کودکان زیر پنج سال[SEP]روند چاقی و اضافه وزن[SEP]در افراد چاق و بویژه کودکان و نوجوانان چاق[SEP]علت دیگر کم بودن تحرک بدنی[SEP]کامپیوتر و گوشی و فعالیتهای نشستن[SEP]کودکان و نوجوانان[SEP]خطر مرگ و میر و ابتلا[SEP]روز چاقی و اضافه وزن[SEP]کودکان و نوجوانان روز[SEP]نظام سلا

# Saving the results

In [48]:
with open('/content/drive/MyDrive/CoPer paper-Models/Results/sbert-WikiNli-DifferentWeights_FineTuned_f300-Ranked_2.pkl', 'wb') as f:
    pickle.dump(results, f)